# Multi-indexing

## Basic multi-index concept

Up to here, we've been working under the assumption that our data is two dimensional. That's why it fits so well in a tabular form, and would do quite well in a simple spreadsheet.

Let's change that! 

In [ ]:
import pandas as pd 
from datetime import datetime
%matplotlib inline 
from matplotlib import pyplot as plt 
plt.rcParams['figure.figsize'] = [16, 4]

Let's take a toy dataset, with some supermarket foods:

In [ ]:
food = pd.read_csv('https://raw.githubusercontent.com/vohcolab/PandaViz-Workshop/main/Pandas/Time%20Series/data/food.csv')
food.head()

Let's think about indexes. So far, we had for each row some unique ID. But in the above case, we have several supermarkets and several products. 

In this case, our prices and quality depend on the **Store** AND on the **Product**. So our index needs to contain both:

In [ ]:
food = food.set_index(['Store', 'Product'])
food = food.sort_index()

food

Multi-indexes are crazy useful, especially when the datasets get bigger. They allow us to go into higher dimentions than two dimentions, and keep the intuitive "tabular" structure. 



Let's learn how to select from this! 

## Selecting in multi-indexes

We need an additional tool to perform selection with multi_indexes, which is `pd.IndexSlice`


In [ ]:
idx = pd.IndexSlice  # <---- convention, get ready to copy paste this a lot 

### loc

Remember loc? Loc is awesome :) 

Remember, loc allows you to select by doing `data.loc[index, columns]`

Now that we have multiple index levels, we will have to use it a bit more explicitly. 

In [ ]:
food

Let's say we wanted to get the price of Apples, at Dingo Poce. 

If this were just a 1 level index, we would just say `data.loc[rows, columns]`. <br>
However, we have two levels of index. So now we would say: <br> `data.loc[idx[first_index_level, second_index_level], columns]`.

In [ ]:
food.loc[idx[: , 'Apples'], 'Price']

Let's answer another question: _What are all the prices at Dingo Poce?_

In [ ]:
# from food, 
# A) get the slice of the index 
# B) where the index_level_0 is 'Dingo Poce', 
# C) and everything on the second level (:)
# D) also, give me all the columns (:)

# food.loc[A[B, C], D]
food.loc[idx['Dingo Poce', :], :]

What if you want the apples from all stores?

In [ ]:
# from food, 
# A) get the slice of the index 
# B) where we want everything from the first level of the index 
# C) and only the apples from the second level of the index 
# D) again, all of the columns 

# food.loc[A[B, C], D]
food.loc[idx[:, 'Apples'], :]

# idx is love. idx is life. 

That works! Let's do a few more, for practice: 

##### _"The apples at Dingo Poce"_

In [ ]:
food.loc[idx['Dingo Poce', 'Apples'], :]

##### _"The price of Bananas, at any store"_

In [ ]:
food.loc[idx[:, 'Bananas'], 'Price']

##### _"The quality of Olives, at Incontinente"_

In [ ]:
food.loc[idx['Incontinente', 'Olives'], 'Quality']

##### _"The quality of Rice and Bananas, at Incontinente"_

In [ ]:
# notice that we can pass a list to the second half of the index slice
food.loc[idx['Incontinente', ['Bananas', 'Rice']], 'Quality']

---- 

### Slicing in multi-indexes 

Sometimes instead of listing the values we want, we might want to take slices. 

Here it will be by alphabetical order (to keep the example simple), but we will see how powerful this is later when we bring in the timeseries. 

From a syntax point of view, slicing means passing `start: end` to our `.loc`, instead of explicitly naming every entry. 

Let's get the quality of the products "from bananas to rice" in Incontinente (from "b" to "r"). This would be problematic if the dataset were not sorted. 

In [ ]:
# Notice the nomenclature "start: end" 
food.loc[idx['Incontinente', 'Bananas': 'Rice'], 'Quality']

### Groupby (group-apply-combine) in multi-indexes

Let's say we want to know the lowest prices at which we can get each product.  

In the case of this tiny dataset, we can just look directly at the data, or select each product at the time and then take the `.min`, but we will use this very simply question to illustrate a powerful concept. 

In [ ]:
food

In [ ]:
food.groupby(level='Product').Price.min()

Eh... that's cool, but a better questions is _"Where should I buy each product, and how much will it cost there?"_

In [ ]:
apples_data = food.loc[idx[:, 'Apples'], :]
apples_data.loc[apples_data.Price == apples_data.Price.min()]

Where is each product cheaper?

In [ ]:
food.groupby(level='Product').Price.idxmin()

So we found out where the food is cheapest. But now I'm worried about the quality... 

We already know where things are cheaper, let's call it `where_stuff_is_cheapest`

In [ ]:
where_stuff_is_cheapest = food.groupby(level='Product').Price.idxmin()

In [ ]:
where_stuff_is_cheapest.head(2)

Now we can use this to select. Notice we don't need `idx`, because our index is already a tuple. 

In [ ]:
# food, where the indexes of the minimum prices are, with the columns Quality and Price 
food.loc[where_stuff_is_cheapest, ['Quality', 'Price']]

So... the minimum price, but only where the quality is higher than 3? 

In [ ]:
# Sigh.... ok, so food where the quality is at least 3...
at_least_3_quality_food = food.loc[food.Quality >=3]

# get the indexes where the lowest price occurs 
index_where_lowest_price = at_least_3_quality_food.groupby(level='Product').Price.idxmin()

# now from the food where the quality is at least 3, 
# get the indexes where the price is minimum, 
# and show both quality and price
at_least_3_quality_food.loc[index_where_lowest_price, ['Quality', 'Price']]

It's so beautiful! 

## Bring in the timeseries! 

Ok, so that was a basic idea of multi-indexing. But that's not what you are here for, you are here for timeseries!

Let's get back to business! 

Let's take all of the stocks from an exchange between 2007 and 2011

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/vohcolab/PandaViz-Workshop/main/Pandas/Time%20Series/data/stocks.csv')

In [ ]:
data.head()

How much data do we have? 

In [ ]:
data.shape

Cool, and we already know how to fix datetimes: 

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])

Now, consider this sentence carefully: 

> _**For each stock, for each date**, we have an Open, High, Low, Close, and Volume_

Which means that we have a multi-index, with both date, and stock! 

In [ ]:
data = data.set_index(['Date', 'Stock'])
data.head()

As mentioned before, we must sort our index:

In [ ]:
data = data.sort_index()
data.head()

##### Select Microsoft (msft):

In [ ]:
data.loc[idx[:, 'msft'], :].head()

##### Select all stocks from May  20th, 2009 

In [ ]:
data.loc[idx['May 20th, 2009', :], :].head(10)

##### Select the Close price of Microsoft, IBM and Ebay, between June 10th and June 14th, 2008?

In [ ]:
close_prices_tech_3 = data.loc[idx['June 10th, 2008':'June 14th, 2008', ['msft', 'ibm' ,'ebay']], 'Close']

close_prices_tech_3

Remember that if you're going to be indexing a certain interval several times it's probably better to create a python slice:

In [ ]:
interval = slice('June 10th, 2008','June 14th, 2008')
data.loc[idx[interval, ['msft', 'ibm' ,'ebay']], 'Close']

#### Working with the index directly 

For fun, we can also ask questions such as: 
> _"What was the average price of stocks on Mondays?"_ 

For this we will use a boolean mask, and a new method, [get_level_values](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&cad=rja&uact=8&ved=0ahUKEwjTisXGrOjaAhXHzRQKHSfJCq8QFggpMAA&url=https%3A%2F%2Fpandas.pydata.org%2Fpandas-docs%2Fstable%2Fgenerated%2Fpandas.Index.get_level_values.html&usg=AOvVaw3XDga7SdrZkID3lvOtiaDz)

In [ ]:
# Make a boolean mask, as to whether the day is Monday. 
# Notice two things: 
# 1. the use of get_level_values(0), to get the index at the dates level
# 2. the use of .weekday_name directly on the index (no need for .dt)
is_monday = data.index.get_level_values(0).day_name() == 'Monday'

# Now use that boolean mask to get the data for Mondays, then get the mean close price
mondays_data = data.loc[is_monday].Close.mean()

print('Mean Monday price: %0.2f' % mondays_data)

## Plotting 

##### Plot the close prices for Microsoft:

Using the previous approach, let's try to make a plot:

In [ ]:
data.loc[idx[:, 'msft'], 'Close'].plot()

## Groupby, and operations 

After this detour, let's get back to our main dataset: 

In [ ]:
data.head(3)

**What's the right way to do this?** _(well, the chapter IS called Groupby...)_

In [ ]:
# group by the stock, and then take the Close, and calculate percentage change 
data.groupby(level='Stock').Close.pct_change().head()

Wait what? Oh, right. On the first day, there isn't a previous day to calculate change. Let's take a look at another day: 

In [ ]:
# exceptionally I'm using the "lazy" way to do loc here, for the sake of simplicity:

data.groupby(level='Stock').Close.pct_change().loc['May 5th 2008'].head()

Let's manually check this. What was the price for `abb` on `2008-05-05`?

In [ ]:
data.loc[idx['May 5th 2008', 'abb'], 'Close']

And on the previous day? 

In [ ]:
data.loc[idx['May 4th 2008', 'abb'], 'Close']

Uuuhh... no data on that day! Which makes it particularly cool that Pandas figures out on its own to look on the previous day where there is some data. 

In [ ]:
data.loc[idx['May 2th 2008': 'May 5th 2008', 'abb'], 'Close']

What is the percentage change between the two days? 

In [ ]:
print('Percentage change between the consecutive days: %0.6f:' % ((22.970 - 22.866) / 22.866))

Which is exactly what we got! 

References: 

- [Somebits](https://www.somebits.com/~nelson/pandas-multiindex-slice-demo.html)
- [Pandas multi-index documentation](http://pandas.pydata.org/pandas-docs/stable/advanced.html#multiindex-advanced-indexing)

-----

### **Summary of the methods we have learnt in this unit:**
* `loc` - loc allows you to select by doing `dataframe.loc[index, columns]`
* We can use idx = `pd.IndexSlice` so that we don't have to use `slice()` all over the place. This will take slices from the index 
* Selecting from multi-index: `dataframe.loc[idx[index_level_0, index_level_1], columns]`
* Groupby different levels: `dataframe.groupby(level=<level_you_want>).mean()` -> Remember you need an aggregation function after groupby. 
* Finding the first occurrence of the min or the max: `idxmin()` or `idxmax()`
* Aggregate by more than one method at a time: `agg()`

### **A few examples:** 

    * An example we have seen: `food.loc[idx['Dingo Poce', :], :]`
    * Slicing multi-index: `food.loc[idx['Incontinente', 'Bananas': 'Rice'], 'Quality']`
    * groupby Product and get the minimum Price: `food.groupby(level='Product').Price.min()`
    * Aggregate by more than one method: `food.groupby(level='Product').agg({'Price':['idxmin', 'min']})`

-----